<a href="https://colab.research.google.com/github/dhyannn/ML-LAB/blob/main/lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import threading
import warnings
warnings.filterwarnings("ignore")
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import fpgrowth

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
df=pd.read_csv("Groceries_dataset.csv")

In [13]:
df

,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk
...,...,...,...
38760,4471,08-10-2014,sliced cheese
38761,2022,23-02-2014,candy
38762,1097,16-04-2014,cake bar
38763,1510,03-12-2014,fruit/vegetable juice


import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(df, *['Member_number'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(df, *['Member_number'], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(df, *['Date', 'Member_number', None], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(df, *['Date', 'count()', None], **{})
chart

In [14]:
df.shape

(38765, 3)

In [17]:
grouped = df.groupby(['Member_number', 'Date'])['itemDescription'].apply(list)
list_of_transactions = grouped.tolist()
list_of_transactions[:5]

[['sausage', 'whole milk', 'semi-finished bread', 'yogurt'],
 ['whole milk', 'pastry', 'salty snack'],
 ['canned beer', 'misc. beverages'],
 ['sausage', 'hygiene articles'],
 ['soda', 'pickled vegetables']]

In [18]:
def fp_growth_algorithm():
    print("Running FP-Growth algorithm...")
    time.sleep(3)
    print("FP-Growth algorithm completed.")
def apriori_algorithm():
    print("Running Apriori algorithm...")
    time.sleep(2)
    print("Apriori algorithm completed.")

def main():
    fp_thread = threading.Thread(target=fp_growth_algorithm)
    apriori_thread = threading.Thread(target=apriori_algorithm)

fp_thread.start()
apriori_thread.start()
fp_thread.join()
apriori_thread.join()

print("Both algorithms have finished.")

NameError: ignored

In [20]:
def fp_growth_algorithm():
    print("Running FP-Growth algorithm...")
    time.sleep(3)
    print("FP-Growth algorithm completed.")

def apriori_algorithm():
    print("Running Apriori algorithm...")
    time.sleep(2)
    print("Apriori algorithm completed.")

if __name__ == "__main__":
    fp_thread = threading.Thread(target=fp_growth_algorithm)
    apriori_thread = threading.Thread(target=apriori_algorithm)


    fp_thread.start()
    apriori_thread.start()


    fp_thread.join()
    apriori_thread.join()

    print("Both algorithms have finished.")

In [21]:
class FPGrowth:
    def __init__(self, min_support=0.1):
        self.min_support = min_support

    def run(self, transactions):
        print(f"Running FP-Growth algorithm with minimum support {self.min_support}...")
        print("FP-Growth algorithm completed.")

class Apriori:
    def __init__(self, min_support=0.1, min_confidence=0.5):
        self.min_support = min_support
        self.min_confidence = min_confidence

    def run(self, transactions):
        print(f"Running Apriori algorithm with minimum support {self.min_support} and minimum confidence {self.min_confidence}...")
        print("Apriori algorithm completed.")

if __name__ == "__main__":
    transactions = [...]

    fp_growth = FPGrowth(min_support=0.2)
    apriori = Apriori(min_support=0.2, min_confidence=0.6)

    fp_growth.run(transactions)
    apriori.run(transactions)

Running FP-Growth algorithm with minimum support 0.2...
FP-Growth algorithm completed.
Running Apriori algorithm with minimum support 0.2 and minimum confidence 0.6...
Apriori algorithm completed.
